In [25]:
#@title Carrega dados ENADE
import pandas as pd
!pip install unidecode
!pip install -q gdown
import os
from unidecode import unidecode
import gdown
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML


def titulo_formatado(texto):
    excecoes = {"de", "da", "do", "das", "dos", "e", "em"}
    texto = texto.replace("(","").replace(")","")
    palavras = texto.lower().split()

    resultado = []
    for i, palavra in enumerate(palavras):
        if palavra in excecoes and i != 0:
            resultado.append(palavra)
        else:
            resultado.append(palavra.capitalize())

    return ' '.join(resultado)

# Carrega dados dos Cursos
id_google = '1yFPQlaMlFUOeChwgPdnnC57Qzb7Nukfy'
url = f"https://drive.google.com/uc?id={id_google}&confirm=t"
output = "cursos.xlsx"  # troque para .xlsx se for Excel

# Faz o download do arquivo
gdown.download(url, output, quiet=False)

# Tenta ler como CSV
try:
    df_cursos = pd.read_excel(output)
except Exception:
    print('Erro ao carregar arquivo')


df_cursos['Curso'] = df_cursos.apply(lambda x: titulo_formatado(x['Curso']), axis=1)

df_cursos['NOME_CURSO_AUX'] =  df_cursos['Grau']+', '+df_cursos['Turno']+', '+df_cursos['Município']
df_cursos['NOME_CURSO_AUX'] = df_cursos['NOME_CURSO_AUX'].str.replace(", -","")

df_cursos['Curso Detalhado'] = df_cursos.apply(lambda x: titulo_formatado(x['Curso']), axis=1)
df_cursos['Curso Detalhado'] = df_cursos['Curso Detalhado']+ ' / '+ df_cursos['Grau']+' / '+df_cursos['Turno']+' / '+df_cursos['Município']
df_cursos.sort_values(by=['Curso Detalhado'],ascending=True)
cursos_id = dict(zip(df_cursos['Curso Detalhado'], df_cursos['Código']))

cursos_ufc = dict(zip(df_cursos['Código'], df_cursos['Curso']))
cursos_complementar_ufc = dict(zip(df_cursos['Código'], df_cursos['NOME_CURSO_AUX']))

# Unidades Acadêmicas
unidades_academicas = sorted(list(df_cursos['Unidade Acadêmica'].unique()))

# Retorna os códigos inep das unidades acadêmicas
def get_codigos_cursos(df_cursos, unidade_academica):
  return df_cursos[df_cursos['Unidade Acadêmica']==unidade_academica]['Código'].unique()

# Carrega dados do ENADE
id_enade = "1B7y2FfC7qdFj_2CVGT9VsUKI0w_2aOYS"
url = f"https://drive.google.com/uc?id={id_enade}&confirm=t"
output = "enade_2014_2023.csv"  # troque para .xlsx se for Excel
# Faz o download do arquivo
gdown.download(url, output, quiet=False)

# Tenta ler como CSV
try:
    df_class_enade = pd.read_csv(output)
except Exception:
    print('Erro ao carregar arquivo')

anos_enade = sorted(df_class_enade['Ano'].unique())

# Carrega dados do ENADE da UFC
id_enade_ufc = "1eHiXdek9yzH9vsJKN57RZfph-UB-F9vn"
url = f"https://drive.google.com/uc?id={id_enade_ufc}&confirm=t"
output = "enade_ufc_2014_2023.csv"  # troque para .xlsx se for Excel

# Faz o download do arquivo
gdown.download(url, output, quiet=False)

# Tenta ler como CSV
try:
    df_class_enade_ufc = pd.read_csv(output)
except Exception:
    print('Erro ao carregar arquivo do enade da ufc')

df_class_enade_ufc = df_class_enade_ufc.merge(df_cursos[['Código','Unidade Acadêmica', 'Modalidade', 'Grau', 'Turno']],left_on=['Código do Curso'], right_on=['Código'], how='left')


Downloading...
From: https://drive.google.com/uc?id=1yFPQlaMlFUOeChwgPdnnC57Qzb7Nukfy&confirm=t
To: /content/cursos.xlsx
100%|██████████| 14.9k/14.9k [00:00<00:00, 17.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1B7y2FfC7qdFj_2CVGT9VsUKI0w_2aOYS&confirm=t
To: /content/enade_2014_2023.csv
100%|██████████| 15.7M/15.7M [00:00<00:00, 155MB/s]
Downloading...
From: https://drive.google.com/uc?id=1eHiXdek9yzH9vsJKN57RZfph-UB-F9vn&confirm=t
To: /content/enade_ufc_2014_2023.csv
100%|██████████| 46.9k/46.9k [00:00<00:00, 52.4MB/s]


,Ano,Código da Área,Área de Avaliação,Código do Curso,Região,UF,Município,IES,Nome da IES,Conceito Enade (Contínuo),...,Nota Padronizada - FG,Nota Padronizada - CE,Nota Bruta - ENADE,Nota Bruta - FG,Nota Bruta - CE,Código,Unidade Acadêmica,Modalidade,Grau,Turno
0,2014,21.0,ARQUITETURA E URBANISMO,13987,Nordeste,CE,FORTALEZA,UFC,UNIVERSIDADE FEDERAL DO CEARÁ,4.075639,...,4.198595,4.034654,58.283803,70.045070,54.363380,13987.0,Instituto de Arquitetura e Urbanismo,Presencial,Bacharelado,Integral
1,2014,4004.0,CIÊNCIA DA COMPUTAÇÃO (BACHARELADO),13980,Nordeste,CE,FORTALEZA,UFC,UNIVERSIDADE FEDERAL DO CEARÁ,4.603923,...,3.849190,4.855500,62.726562,70.318750,60.195833,13980.0,Centro de Ciências,Presencial,Bacharelado,Integral
2,2014,1601.0,CIÊNCIAS BIOLÓGICAS (BACHARELADO),313974,Nordeste,CE,FORTALEZA,UFC,UNIVERSIDADE FEDERAL DO CEARÁ,3.409534,...,3.303800,3.444778,54.571875,63.250000,51.679167,313974.0,Centro de Ciências,Presencial,Bacharelado,Integral
3,2014,1602.0,CIÊNCIAS BIOLÓGICAS (LICENCIATURA),13974,Nordeste,CE,FORTALEZA,UFC,UNIVERSIDADE FEDERAL DO CEARÁ,3.493726,...,2.755045,3.739954,54.731250,57.300000,53.875000,13974.0,Centro de Ciências,Presencial,Licenciatura,Integral
4,2014,5401.0,CIÊNCIAS SOCIAIS (BACHARELADO),313982,Nordeste,CE,FORTALEZA,UFC,UNIVERSIDADE FEDERAL DO CEARÁ,2.310247,...,2.211772,2.343072,45.146667,54.626667,41.986667,313982.0,Centro de Humanidades,Presencial,Bacharelado,Integral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,2023,12.0,MEDICINA,13996,Nordeste,CE,FORTALEZA,UFC,UNIVERSIDADE FEDERAL DO CEARÁ,3.969675,...,4.439854,3.812949,72.102932,75.211728,71.066667,13996.0,Faculdade de Medicina,Presencial,Bacharelado,Integral
240,2023,12.0,MEDICINA,54490,Nordeste,CE,SOBRAL,UFC,UNIVERSIDADE FEDERAL DO CEARÁ,3.849046,...,3.989201,3.802327,71.457333,72.809333,71.006667,54490.0,Campus de Sobral,Presencial,Bacharelado,Integral
241,2023,6.0,ODONTOLOGIA,99308,Nordeste,CE,SOBRAL,UFC,UNIVERSIDADE FEDERAL DO CEARÁ,4.298138,...,4.337791,4.284921,64.058784,63.472973,64.254054,99308.0,Campus de Sobral,Presencial,Bacharelado,Integral
242,2023,6.0,ODONTOLOGIA,13997,Nordeste,CE,FORTALEZA,UFC,UNIVERSIDADE FEDERAL DO CEARÁ,4.246793,...,4.805249,4.060641,63.783333,67.300000,62.611111,13997.0,"Faculdade de Farmácia, Odontologia e Enfermagem",Presencial,Bacharelado,Integral


In [31]:
#@title Gráfico dos conceitos ENADE do último triênio
def gerar_grafico(df, rotulo_anos='cursos', s_coluna_indicador='Conceito Enade (Faixa)', s_title=None, file_name=None, show_fig=False):
  # Total de Alunos
  total = df[rotulo_anos].sum()

  cores_faixas = {
    '1': '#d73027',
    '2': '#fc8d59',
    '3': '#fee08b',
    '4': '#91cf60',
    '5': '#4575b4'
    # '5': '#1a9850'
  }

  cores = [cores_faixas[c] for c in df[s_coluna_indicador].unique()]
  # Aumentar o tamanho da figura (altura e largura)
  fig_height = 2.0
  if s_title:
    fig_height = 2.32
  fig, ax = plt.subplots(figsize=(14, fig_height))  # altura maior

  start = 0
  for i, row in df.iterrows():
      ax.barh(0, row[rotulo_anos], left=start, color=cores[i], height=0.4)  # height ajustado

      # Texto percentual centralizado
      if row['%'] >= 1:
          ax.text(
              start + row[rotulo_anos] / 2,
              0,
              f"{row['%']:.1f}%",
              va='center',
              ha='center',
              color='white',
              fontsize=12,
              fontweight='bold'
          )


      start += row[rotulo_anos]

  # Ajustes
  ax.set_xlim(0, total)
  ax.set_yticks([])
  if total<10:
    plt.gca().xaxis.set_major_locator(MultipleLocator(1))

  f_title = f"(Cursos {df[rotulo_anos].sum():,})".replace(",",".")
  if s_title!=None:
    f_title = f"{s_title} {f_title}"
  ax.set_title(f_title, fontsize=16)

  # Legenda com totais
  handles = [plt.Rectangle((0, 0), 1, 1, color=cores[i]) for i in range(len(df))]
  labels = [
      f"{df[s_coluna_indicador][i]} ({df[rotulo_anos][i]:,})".replace(",", ".")
      for i in range(len(df))
  ]
  ax.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, -0.25), ncol=6, fontsize=10)

  plt.tight_layout()

  # Salvar ou exibir
  if file_name:
      plt.savefig(f"{file_name}", format="png", bbox_inches='tight', dpi=300)

  if show_fig:
      plt.show()

def display_filtrar_ultimos_enades():
  run = widgets.Button(description="Gerar")
  wUnidades = widgets.Dropdown(
        options=['']+unidades_academicas,
        # value=None,
        description=f'Unidade Acadêmica',
        style={'description_width': '180px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )

  output = widgets.Output()
  display(
          wUnidades,
          run,output)

  def on_button_run_clicked(_):
    output.clear_output()
    with output:
      UA = wUnidades.value
      ultimos_anos = sorted(df_class_enade_ufc['Ano'].unique())[-3:]
      if UA=='':
        df_class_enade_ufc_ultimo_trienio = df_class_enade_ufc[(df_class_enade_ufc['Ano'].isin(ultimos_anos))].copy()
      else:
        df_class_enade_ufc_ultimo_trienio = df_class_enade_ufc[(df_class_enade_ufc['Ano'].isin(ultimos_anos))
          &(df_class_enade_ufc['Código do Curso'].isin(get_codigos_cursos(df_cursos,UA)))].copy()

      if len(df_class_enade_ufc_ultimo_trienio)==0:
        print("Nenhuma avaliação foi encontrada!")
      else:
        df_enade_resumo = df_class_enade_ufc_ultimo_trienio[df_class_enade_ufc_ultimo_trienio['Conceito Enade (Faixa)']!='SC'].groupby(by='Conceito Enade (Faixa)').agg(cursos=('Código da Área','count')).reset_index()

        df_enade_resumo['%'] = (df_enade_resumo['cursos'] / df_enade_resumo['cursos'].sum()) * 100

        total_row = pd.DataFrame({
            'Conceito Enade (Faixa)': ['Total'],
            'cursos': [df_enade_resumo['cursos'].sum()],
            '%': [df_enade_resumo['%'].sum()]
        })

        df_enade_resumo['%'] = df_enade_resumo['%'].round(2)

        # Adicionar ao DataFrame
        df_enade_resumo = pd.concat([df_enade_resumo, total_row], ignore_index=True)

        filename_save_enade_ultimos_trienio = f"enade_último_triênio_ufc.png"
        gerar_grafico(df_enade_resumo[:-1], s_title=f"Conceito ENADE - Triênio {', '.join(str(a) for a in ultimos_anos)}",show_fig=True, file_name=filename_save_enade_ultimos_trienio)

  run.on_click(on_button_run_clicked)

display_filtrar_ultimos_enades()

Dropdown(description='Unidade Acadêmica', layout=Layout(width='600px'), options=('', 'Campus de Crateús', 'Cam…

Button(description='Gerar', style=ButtonStyle())

Output()

In [3]:
#@title Cards do ENADE
def titulo_formatado(texto):
    excecoes = {"de", "da", "do", "das", "dos", "e", "em"}
    texto = texto.replace("(","").replace(")","")
    palavras = texto.lower().split()

    resultado = []
    for i, palavra in enumerate(palavras):
        if palavra in excecoes and i != 0:
            resultado.append(palavra)
        else:
            resultado.append(palavra.capitalize())

    return ' '.join(resultado)

def gerar_card(df, curso, curso_modalidade_municipio, titulo_universidade='Universidade Federal do Ceará', titulo_avaliacao='ENADE', coluna_conceito='Conceito Enade (Faixa)',file_name=None, show_fig=False, max_anos=3):

    # Gera no máximo as três últimas avaliações
    df = df.tail(min(3, max_anos)).reset_index(drop=True).copy()
    t_anos = len(df)

    df = df.iloc[-1*max_anos:].copy()

    # Cores
    fundo_azul = '#1E237E'
    branco = '#ffffff'
    azul_escuro = '#262A70'
    azul_claro = '#0099FF'
    verde_claro = '#A6F4A6'

    # Sempre formato quadrado (ideal para Instagram)
  # Tamanho do gráfico baseado no número de anos
    if t_anos == 1:
        fig, ax = plt.subplots(figsize=(6, 6))  # Formato quadrado
    elif t_anos == 2:
        fig, ax = plt.subplots(figsize=(9, 6))  # Formato quadrado
    else:
        fig, ax = plt.subplots(figsize=(12, 6))  # Original

    # fig, ax = plt.subplots(figsize=(6, 6))
    fig.patch.set_facecolor(fundo_azul)
    ax.set_facecolor(fundo_azul)
    ax.axis('off')

    # Título
    plt.text(0.5, 0.9, titulo_universidade, fontsize=22, fontweight='bold', color=branco, ha='center')
    plt.text(0.5, 0.83, curso, fontsize=22, fontweight='bold', color=branco, ha='center')
    plt.text(0.5, 0.76, curso_modalidade_municipio, fontsize=16, fontweight='bold', color=branco, ha='center')

    # Posição inicial e espaçamento baseados em t_anos
    if t_anos == 3:
        card_width = 0.25
        spacing = 0.05
        total_width = 3 * card_width + 2 * spacing
        start_x = (1 - total_width) / 2
    elif t_anos == 2:
        card_width = 0.35
        spacing = 0.08
        total_width = 2 * card_width + spacing
        start_x = (1 - total_width) / 2
    else:  # Apenas 1 card
        card_width = 0.6
        start_x = (1 - card_width) / 2
        spacing = 0  # irrelevante

    for i, row in df.iterrows():
        x = start_x + i * (card_width + spacing)

        # Caixa branca (card)
        rect = patches.FancyBboxPatch((x, 0.1), card_width, 0.55,
                                      boxstyle="round,pad=0.02,rounding_size=0.0",
                                      edgecolor='none', facecolor=branco)
        ax.add_patch(rect)

        # Título do card (ano)
        ax.text(x + card_width / 2, 0.65, f'{titulo_avaliacao} {row["Ano"]}', fontsize=16, fontweight='bold',
                color=azul_escuro, ha='center', bbox=dict(boxstyle="round,pad=0.3", facecolor=verde_claro, edgecolor='none'))

        # Conceito
        ax.text(x + card_width / 2, 0.53, 'CONCEITO', fontsize=20, fontweight='bold', color=azul_escuro, ha='center')
        ax.text(x + card_width / 2, 0.45, f'{row[coluna_conceito]}', fontsize=28, fontweight='bold', color=azul_escuro, ha='center')

        # Classificação Brasil
        ax.text(x + card_width / 2, 0.35, f'Brasil: {row["Classificação no Brasil"]}º de {row["Cursos no Brasil"]}', fontsize=14, fontweight='bold',
                color=azul_claro, ha='center', bbox=dict(boxstyle="round,pad=0.3", facecolor=branco, edgecolor=azul_claro, linewidth=2))

        # Classificação Região
        ax.text(x + card_width / 2, 0.25, f'Nordeste: {row["Classificação na Região"]}º de {row["Cursos na Região"]}', fontsize=14, fontweight='bold',
                color=azul_claro, ha='center', bbox=dict(boxstyle="round,pad=0.3", facecolor=branco, edgecolor=azul_claro, linewidth=2))

        # Classificação Estado
        ax.text(x + card_width / 2, 0.15, f'Ceará: {row["Classificação no Estado"]}º de {row["Cursos no Estado"]}', fontsize=14, fontweight='bold',
                color=azul_claro, ha='center', bbox=dict(boxstyle="round,pad=0.3", facecolor=branco, edgecolor=azul_claro, linewidth=2))

    # Salvar ou exibir
    if file_name:
        plt.savefig(f"{file_name}", format="png", bbox_inches='tight', dpi=300)

    if show_fig:
        plt.show()

    plt.close()

def display_filtrar_coortes():
  run = widgets.Button(description="Gerar")
  wCURSOS = widgets.Dropdown(
        options=cursos_id,
        # value=None,
        description=f'Curso',
        style={'description_width': '80px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )
  wAnos = widgets.Dropdown(
        options=[1,2,3],
        value=3,
        description=f'Últimos anos',
        style={'description_width': '80px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )

  output = widgets.Output()
  display(
          wCURSOS,wAnos,
          run,output)

  def on_button_run_clicked(_):
    output.clear_output()
    with output:
      CO_CURSO = wCURSOS.value
      max_anos = wAnos.value
      df_class_enade_ufc_area_curso = df_class_enade_ufc[df_class_enade_ufc['Código do Curso']==CO_CURSO]
      if len(df_class_enade_ufc_area_curso)==0:
        print("Nenhuma avaliação foi encontrada!")
      else:
        gerar_card(df_class_enade_ufc_area_curso,cursos_ufc[CO_CURSO],cursos_complementar_ufc[CO_CURSO],file_name='enade.png',show_fig=True,max_anos=max_anos)

  run.on_click(on_button_run_clicked)

display_filtrar_coortes()

Dropdown(description='Curso', layout=Layout(width='600px'), options={'Biotecnologia / Bacharelado / Integral /…

Dropdown(description='Últimos anos', index=2, layout=Layout(width='600px'), options=(1, 2, 3), style=Descripti…

Button(description='Gerar', style=ButtonStyle())

Output()

In [4]:
#@title Indicadores (Nota Padronizada) do ENADE
def gerar_grafico_enade(df_class_enade_ufc_curso, curso=None, curso_modalidade_municipio=None, universidade='Universidade Federal do Ceará', file_name=None, show_fig=False, max_anos=3):
  df_class_enade_ufc_curso = df_class_enade_ufc_curso.iloc[-1*min(3,max_anos):]
  anos = df_class_enade_ufc_curso['Ano'].unique()
  n_anos = len(anos)
  if curso==None:
    curso = titulo_formatado(df_class_enade_ufc_curso['Área de Avaliação'].unique()[0])
  if curso_modalidade_municipio==None:
    curso_modalidade_municipio = titulo_formatado(df_class_enade_ufc_curso['Município'].unique()[0])

  dados = {
      'Formação Geral (25%)':[],
      'Formação Específica (75%)':[],
      'Enade':[],
  }

  # Deslocamentos verticais para cada ano
  for i, ano in enumerate(anos):
    enade_continuo = df_class_enade_ufc_curso.iloc[i]['Conceito Enade (Contínuo)']
    fg = df_class_enade_ufc_curso.iloc[i]['Nota Padronizada - FG']
    ce = df_class_enade_ufc_curso.iloc[i]['Nota Padronizada - CE']

    dados['Formação Geral (25%)'].append(fg)
    dados['Formação Específica (75%)'].append(ce)
    dados['Enade'].append(enade_continuo)

  categorias = list(dados.keys())
  n_grupos = len(anos)
  n_categorias = len(categorias)

  # Largura das barras
  if n_anos==3:
    bar_height = 0.25
  elif n_anos==2:
    bar_height = 0.35
  elif n_anos==1:
    bar_height = 0.45
  indice_base = np.arange(n_categorias)

  # Plot
  plt.figure(figsize=(10, 6))

  # Deslocamentos verticais para cada ano
  for i, ano in enumerate(anos):
    valores = [dados[cat][i] for cat in categorias]
    deslocamento = bar_height * (i - (n_anos - 1) / 2)  # centraliza automaticamente
    plt.barh(indice_base + deslocamento, valores, height=bar_height, label=str(ano))

  # Eixos e rótulos
  plt.yticks(indice_base, categorias)
  # Linha tracejada vertical no valor 5
  plt.axvline(x=0.945, color='red', linestyle='--', linewidth=2, label='Conceito 2')
  plt.axvline(x=1.945, color='pink', linestyle='--', linewidth=2, label='Conceito 3')
  plt.axvline(x=2.945, color='yellow', linestyle='--', linewidth=2, label='Conceito 4')
  plt.axvline(x=3.945, color='blue', linestyle='--', linewidth=2, label='Conceito 5')

  # Estética
  plt.xlabel('Nota Contínua de 0 a 5')
  plt.xticks([])


  plt.title(f"{universidade}\n{curso}\n{curso_modalidade_municipio}", fontsize=22, fontweight='bold', ha='center')
  # plt.text(0.5, 0.82, , fontsize=22, fontweight='bold', color=branco, ha='center')

  plt.xlim(0, 5)
  # plt.xlim(0, max(max(v) for v in dados.values()) + 1)
  plt.grid(axis='x', linestyle=':', alpha=0.5)

  # Legenda abaixo
  plt.legend(loc='upper center', bbox_to_anchor=(0.20, -0.05), ncol=7)

  plt.tight_layout()
  # Salvar ou exibir
  if file_name:
      plt.savefig(f"{file_name}", format="png", bbox_inches='tight', dpi=300)

  if show_fig:
      plt.show()

  plt.close()


def display_enade_prova():
  run = widgets.Button(description="Gerar")
  wCURSOS = widgets.Dropdown(
        options=cursos_id,
        # value=None,
        description=f'Curso',
        style={'description_width': '80px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )
  wAnos = widgets.Dropdown(
        options=[1,2,3],
        value=3,
        description=f'Últimos anos',
        style={'description_width': '80px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )

  output = widgets.Output()
  display(
          wCURSOS,wAnos,
          run,output)

  def on_button_run_clicked(_):
    output.clear_output()
    with output:
      CO_CURSO = wCURSOS.value
      max_anos = wAnos.value
      df_class_enade_ufc_area_curso = df_class_enade_ufc[df_class_enade_ufc['Código do Curso']==CO_CURSO]
      if len(df_class_enade_ufc_area_curso)==0:
        print("Nenhuma avaliação foi encontrada!")
      else:
        # gerar_card(df_class_enade_ufc_area_curso,cursos_ufc[CO_CURSO],cursos_complementar_ufc[CO_CURSO],file_name='enade.png',show_fig=True,max_anos=3)
        gerar_grafico_enade(df_class_enade_ufc_area_curso,curso=cursos_ufc[CO_CURSO],curso_modalidade_municipio=cursos_complementar_ufc[CO_CURSO],max_anos=max_anos,show_fig=True)

  run.on_click(on_button_run_clicked)

display_enade_prova()

Dropdown(description='Curso', layout=Layout(width='600px'), options={'Biotecnologia / Bacharelado / Integral /…

Dropdown(description='Últimos anos', index=2, layout=Layout(width='600px'), options=(1, 2, 3), style=Descripti…

Button(description='Gerar', style=ButtonStyle())

Output()

In [5]:
#@title Indicadores (Nota Bruta) do ENADE
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator


def gerar_grafico_indicadores(categorias,valores,ano, limites_conceitos, curso, curso_modalidade_municipio, universidade='Universidade Federal do Ceará', file_name=None, show_fig=False, limite_max=100):
  cores_conceitos = {
      1.945: 'red',  # Conceito 2
      2.945: 'pink',  # Conceito 3
      3.945: 'yellow',  # Conceito 4
      4.945: 'blue'   # Conceito 5
  }

  fig, ax = plt.subplots(figsize=(10, 5))
  y_pos = np.arange(len(categorias))
  bar_height = 0.55

  # Barras
  ax.barh(y_pos, valores, height=bar_height, color='LightSkyBlue')

  # Linhas pontilhadas por barra
  for i, categoria in enumerate(categorias):
      limites = limites_conceitos[categoria]
      ax.vlines(limites[0], i - bar_height/2, i + bar_height/2, colors='red', linestyles='dotted', linewidth=3)
      ax.vlines(limites[1], i - bar_height/2, i + bar_height/2, colors='orange', linestyles='dotted', linewidth=3)
      ax.vlines(limites[2], i - bar_height/2, i + bar_height/2, colors='green', linestyles='dotted', linewidth=3)
      ax.vlines(limites[3], i - bar_height/2, i + bar_height/2, colors='blue', linestyles='dotted', linewidth=3)

      # for lim in limites:
      #     cor = cores_conceitos.get(lim, 'gray')
      #     ax.vlines(lim, i - bar_height/2, i + bar_height/2, colors=cor, linestyles='dotted', linewidth=2)

  # Eixos e título
  ax.set_yticks(y_pos)
  ax.set_yticklabels(categorias)
  ax.set_xlim(0, limite_max)

  if limite_max==100:
    plt.gca().xaxis.set_major_locator(MultipleLocator(5))

  ax.set_xlabel(f'Nota (0–{limite_max})')
  # ax.set_title('Notas por Indicador com Limites de Conceito Coloridos')
  ax.set_title(f"{universidade}\n{curso}\n{curso_modalidade_municipio}", fontsize=18, fontweight='bold', ha='center')

  # Legenda personalizada
  legenda_conceitos = [
      Line2D([0], [0], color='red', linestyle='dotted', linewidth=3, label='Conceito 2'),
      Line2D([0], [0], color='orange', linestyle='dotted', linewidth=3, label='Conceito 3'),
      Line2D([0], [0], color='green', linestyle='dotted', linewidth=3, label='Conceito 4'),
      Line2D([0], [0], color='blue', linestyle='dotted', linewidth=3, label='Conceito 5'),
      # Line2D([0], [0], color='LightSkyBlue', linestyle='-', linewidth=3, label=f'Ano {ano}'),
      Patch(facecolor='LightSkyBlue', label=f'Ano {ano}')
  ]

  # Legenda abaixo e centralizada
  ax.legend(handles=legenda_conceitos, loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=5)

  # plt.grid(axis='x', linestyle=':', alpha=0.4)
  plt.tight_layout()
  if file_name:
      plt.savefig(f"{file_name}", format="png", bbox_inches='tight', dpi=300)
  if show_fig:
      plt.show()


def conceito_enade(nota_continua):
    if nota_continua < 0.945:
        return 1
    elif nota_continua < 1.945:
        return 2
    elif nota_continua < 2.945:
        return 3
    elif nota_continua < 3.945:
        return 4
    else:
        return 5

def get_limites_notas(df, categoria, categoria_padronizada, ano, area):
  df_limite_categoria = df[(df['Ano']==ano)&(df['Código da Área']==area)][[categoria_padronizada, categoria]].copy()
  df_limite_categoria['conceito']= df_limite_categoria.apply(lambda x: conceito_enade(x[categoria_padronizada]),axis=1)
  df_limite_categoria.sort_values(by=categoria_padronizada,inplace=True)
  df_notas_minimas = df_limite_categoria.groupby(by=['conceito']).nth(0)
  return list(df_notas_minimas[categoria])[1:]


def gerar_grafico_indicadores_curso_enade(df_class_enade, CO_CURSO, ano=None,curso=None,curso_modalidade_municipio=None, file_name=None, show_fig=False):
  if ano==None:
    df_class_enade_curso = df_class_enade[df_class_enade['Código do Curso']==CO_CURSO][-1:]
    ano = df_class_enade_curso.iloc[0]['Ano']
  else:
    df_class_enade_curso = df_class_enade[(df_class_enade['Código do Curso']==CO_CURSO)&(df_class_enade['Ano']==ano)]
  df_class_enade_curso = df_class_enade_curso[-1:]
  area = df_class_enade_curso.iloc[0]['Código da Área']
  fg = df_class_enade_curso.iloc[0]['Nota Bruta - FG']
  ce = df_class_enade_curso.iloc[0]['Nota Bruta - CE']
  enade = df_class_enade_curso.iloc[0]['Nota Bruta - ENADE']
  categorias = ['Formação Geral (25%)', 'Formação Específica (75%)','Enade']
  valores = [fg,ce,enade]
  limites_conceitos = {
      'Formação Geral (25%)': get_limites_notas(df_class_enade, 'Nota Bruta - FG', 'Nota Padronizada - FG' ,ano, area),
      'Formação Específica (75%)': get_limites_notas(df_class_enade, 'Nota Bruta - CE', 'Nota Padronizada - CE' ,ano, area),
      'Enade': get_limites_notas(df_class_enade, 'Nota Bruta - ENADE', 'Conceito Enade (Contínuo)' ,ano, area)
  }
  gerar_grafico_indicadores(categorias,valores,ano,limites_conceitos,curso=curso,curso_modalidade_municipio=curso_modalidade_municipio, file_name=file_name, show_fig=show_fig,limite_max=100)


def display_enade_notas_brutas():
  run = widgets.Button(description="Gerar")
  wCURSOS = widgets.Dropdown(
        options=cursos_id,
        # value=None,
        description=f'Curso',
        style={'description_width': '80px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )

  wAno = widgets.Dropdown(
        options=anos_enade,
        value=anos_enade[-1],
        description=f'Ano',
        style={'description_width': '80px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )



  output = widgets.Output()
  display(
          wCURSOS, wAno,
          run,output)

  def on_button_run_clicked(_):
    output.clear_output()
    with output:
      ano = wAno.value
      CO_CURSO = wCURSOS.value
      df_class_enade_ufc_area_curso = df_class_enade[(df_class_enade['Código do Curso']==CO_CURSO)&(df_class_enade['Ano']==ano)]
      if len(df_class_enade_ufc_area_curso)==0:
        print("Nenhuma avaliação foi encontrada para este curso e ano ENADE!")
      else:
        gerar_grafico_indicadores_curso_enade(df_class_enade, CO_CURSO, ano=ano,curso=cursos_ufc[CO_CURSO],curso_modalidade_municipio=cursos_complementar_ufc[CO_CURSO],show_fig=True)

  run.on_click(on_button_run_clicked)

display_enade_notas_brutas()

Dropdown(description='Curso', layout=Layout(width='600px'), options={'Biotecnologia / Bacharelado / Integral /…

Dropdown(description='Ano', index=8, layout=Layout(width='600px'), options=(np.int64(2014), np.int64(2015), np…

Button(description='Gerar', style=ButtonStyle())

Output()